In [21]:
import openai
import os
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from IPython.display import display, Markdown, Latex, HTML, JSON 
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Open the JSON file
with open('/Users/edwin/Repositories/api_keys.json') as file:
    data = json.load(file)

# Access the API keys
openai.api_key= data['apiKeys']['OpenAI']

In [3]:
data_scientist_template = """You are a very smart data scientist. \
Your work is characterized by creativity, collaboration, problem-solving,\
and a deep understanding of mathematics, statistics, and programming. \
You are so good because you are able to break down\
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


data_engineer = """You are a very good data engineer. \
You are great at answering questions about how to build systems for collecting, storing and analyzing data at scale. \
You are so good because you are able to explain everything in a very structured and easy to understand manner.\
Also you know about different softwares that can be used\
and give the best suggestions based in your experience. 

Here is a question:
{input}"""

scrum_master = """You are a very experienced Scrum Master. As this, you are an agile project management expert. \
Your role involves solving questions about how to facilitate teamwork, \ 
promote agile practices, and remove obstacles. 

Here is a question:
{input}"""


project_manager_template = """ You are an accomplished project manager. \
You possess a fervent enthusiasm for innovation, teamwork, strategic vision,\
self-assurance, adeptness in troubleshooting, comprehension of methodologies\
and workflows, and outstanding interpersonal abilities. Your expertise extends\
beyond mere coding questions, as you excel in coordinating and orchestrating projects effectively. \
Your proficiency lies in articulating solutions to complex problems through sequential, \
machine-interpretable steps while meticulously selecting approaches that strike a \
harmonious equilibrium between time and space complexities.

Here is a question:
{input}"""

In [5]:
prompt_infos = [
    {
        "name": "data science", 
        "description": "Good for answering questions about mathematics, statistics, and programming", 
        "prompt_template": data_scientist_template
    },
    {
        "name": "data engineering", 
        "description": "Good for answering data engineering questions", 
        "prompt_template": data_engineer
    },
    {
        "name": "scrum", 
        "description": "Good for answering scrum questions", 
        "prompt_template": scrum_master
    },
    {
        "name": "project management", 
        "description": "Good for answering project management questions", 
        "prompt_template": project_manager_template
    }
]

In [22]:
llm = ChatOpenAI(temperature=0,openai_api_key=data['apiKeys']['OpenAI'])
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [23]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [24]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [25]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [26]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), #This allows the system to know which chain to choose
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [27]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [28]:
chain.run("A retail company called Real Plaza has gave me a project of building an application in power apps. They want to build their apps as time efficient as possible. I was thinking of using sharepoint tables as data source. Do you think it is the best idea?")



> Entering new  chain...


c:\Users\edwin\anaconda3\envs\data_analysis\lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


project management: {'input': 'A retail company called Real Plaza has given me a project of building an application in power apps. They want to build their apps as time efficient as possible. I was thinking of using SharePoint tables as a data source. Do you think it is the best idea?'}
> Finished chain.


'As an AI language model, I cannot provide a definitive answer as I lack context and information about the specific requirements and constraints of the project. However, as a general suggestion, using SharePoint tables as a data source can be a good idea as it provides a centralized and secure location for data storage and management. It also allows for easy integration with other Microsoft applications such as Power Apps. However, it is important to consider other factors such as scalability, performance, and user experience when making a decision. It may be helpful to consult with other experts or conduct further research to determine the best approach for the project.'

In [29]:
chain.run("Do you have the idea of scale for using sharepoint lists? What are the difference with dataverse?")



> Entering new  chain...
None: {'input': 'Do you have the idea of scale for using sharepoint lists? What are the difference with dataverse?'}
> Finished chain.


'As an AI language model, I do not have personal experience or opinions. However, I can provide you with some information on the idea of scale for using SharePoint lists and the differences with Dataverse.\n\nSharePoint lists are a popular way to store and manage data in SharePoint. They are easy to set up and can be customized to fit specific business needs. SharePoint lists can handle a large amount of data, but their performance can be affected if the list becomes too large or complex.\n\nDataverse, on the other hand, is a cloud-based data platform that is designed to handle large amounts of data and complex business logic. It is a more robust solution than SharePoint lists and is often used for enterprise-level applications. Dataverse provides more advanced features such as data validation, business rules, and workflows.\n\nIn summary, SharePoint lists are suitable for small to medium-sized data sets, while Dataverse is designed for larger and more complex data sets.'

In [31]:
chain.run("A retail company called Real Plaza has gave me a project of building an application in power apps. They want to build their apps as time efficient as possible. Now, how can I start this project")



> Entering new  chain...
project management: {'input': 'A retail company called Real Plaza has given me a project of building an application in power apps. They want to build their apps as time efficient as possible. Now, how can I start this project'}
> Finished chain.


'?\n\nTo start this project, you should first gather all the necessary information about the requirements and objectives of the application from Real Plaza. This includes understanding the specific features and functionalities they want in the app, the target audience, and the timeline for completion.\n\nNext, you should analyze the current processes and workflows of the retail company to identify areas where the app can improve efficiency and save time. This will help you determine the scope of the project and prioritize the features that are most important.\n\nOnce you have a clear understanding of the requirements and scope, you can start designing the app using Power Apps. This involves creating a user-friendly interface, integrating data sources, and testing the app to ensure it meets the requirements and functions as intended.\n\nThroughout the project, it is important to communicate regularly with Real Plaza to ensure that the app is meeting their expectations and to address any